# Setting Extraction Agent 테스트 (Production Level)

장소/배경 추출 에이전트 (Level 1) 테스트 노트북

## 역할: "무대 감독" (Stage Director)
- 배경, 조명, 분위기 담당
- Neo4j Location 노드 생성용 데이터
- 이미지 생성 AI용 background 프롬프트

> **Note**: Neo4j 저장 및 Image Prompt 조립은 Spring Boot에서 처리됩니다.

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
SAMPLE_STORY = """
어두운 숲 속에서 서진은 검을 쥐고 있었다. 키가 크고 검은 머리카락을 가진 그는 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
오래된 거목들이 하늘을 가리고 있었고, 땅에는 짙은 안개가 깔려 있었다. 달빛이 나뭇잎 사이로 희미하게 비추고 있었다.

"우리가 이곳에서 그를 만날 수 있을까?" 서진의 옆에 서 있던 하나가 물었다.
하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다. 그녀는 하얀 치료사 로브를 입고 있었다.

갑자기 나무 사이에서 그림자가 나타났다. 그것은 이민호였다. 한때는 서진의 가장 친한 친구였지만,
지금은 그들의 마을을 배신한 적이 되어버렸다. 그의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. 검은 갑옷을 입은 그는 예전보다 더 날카로워 보였다.
서진은 용감하게 앞으로 나섰다. 긴장감이 최고조에 달했다. "왜 마을을 배신했어, 민호?"

박서연이라는 젊은 여성이 숲에서 뛰어나왔다. 그녀는 마을의 정탐꾼으로, 빠르고 민첩했다.
"경비병들이 오고 있어요! 빨리 결정해야 해요!" 그녀의 목소리에는 급박함이 묻어났다.
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
            "extracted_settings": [], "consistency_report": {}, "retry_count": 0, 
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

def pretty_print(title, data):
    print(f"\n{'='*70}")
    print(f"📋 {title}")
    print(f"{'='*70}")
    print(json.dumps(data, ensure_ascii=False, indent=2))

## 1. Setting Extraction 테스트

In [3]:
from app.agents.extraction.setting import setting_extraction_node

async def test_setting_extraction():
    print("🏰 Setting Extraction (Production Level) 테스트...")
    state = create_base_state()
    return await setting_extraction_node(state)

result = run_async(test_setting_extraction())
pretty_print("Setting 추출 결과", result)

🏰 Setting Extraction (Production Level) 테스트...

📋 Setting 추출 결과
{
  "extracted_settings": [
    {
      "setting_id": "loc_dark_forest",
      "name": "Dark Forest",
      "location_type": "forest",
      "static_visual_prompt": "Towering ancient trees with twisted, gnarled trunks and branches reaching up to block out the sky, thick ground fog obscuring the forest floor, pale moonlight filtering weakly through the dense canopy casting long shadows between the trees, moss-covered rocks and fallen leaves scattered on the ground",
      "time_of_day": "night",
      "lighting_description": "dim, hazy moonlight",
      "atmosphere_keywords": "ominous, mysterious, foreboding",
      "weather_condition": "foggy",
      "static_objects": [
        "ancient twisted trees",
        "thick ground fog",
        "moss-covered rocks",
        "fallen leaves"
      ],
      "is_primary_location": true,
      "story_significance": "Site of the confrontation",
      "visual_background": "Towering anci

## 2. Setting 분석 - 추출 결과 확인

In [4]:
settings = result.get("extracted_settings", [])

print("\n" + "="*70)
print("📊 Setting 분석 (Production Level)")
print("="*70)

for setting in settings:
    print(f"\n🏰 [{setting.get('setting_id', 'N/A')}] {setting.get('name', 'Unknown')}")
    print(f"   Type: {setting.get('location_type', 'N/A')}")
    print(f"   Time: {setting.get('time_of_day', 'N/A')}")
    print(f"   Atmosphere: {setting.get('atmosphere', 'N/A')}")
    
    print(f"\n   🎨 Visual Background (이미지 생성용):")
    print(f"      \"{setting.get('visual_background', 'N/A')}\"")
    
    if setting.get('lighting'):
        print(f"   💡 Lighting: {setting.get('lighting')}")
    if setting.get('weather'):
        print(f"   🌤️ Weather: {setting.get('weather')}")
    if setting.get('notable_features'):
        print(f"   ✨ Features: {setting.get('notable_features')}")


📊 Setting 분석 (Production Level)

🏰 [loc_dark_forest] Dark Forest
   Type: forest
   Time: night
   Atmosphere: ominous, mysterious, foreboding

   🎨 Visual Background (이미지 생성용):
      "Towering ancient trees with twisted, gnarled trunks and branches reaching up to block out the sky, thick ground fog obscuring the forest floor, pale moonlight filtering weakly through the dense canopy casting long shadows between the trees, moss-covered rocks and fallen leaves scattered on the ground"
   💡 Lighting: dim, hazy moonlight
   🌤️ Weather: foggy
   ✨ Features: ['ancient twisted trees', 'thick ground fog', 'moss-covered rocks', 'fallen leaves']

🏰 [loc_village] The Village
   Type: village
   Time: unknown
   Atmosphere: peaceful, homely, rustic

   🎨 Visual Background (이미지 생성용):
      "Rustic medieval fantasy village, small wooden houses with thatched straw roofs lining narrow cobblestone streets, warm orange lantern light glowing from the windows"
   💡 Lighting: warm ambient lantern light
  

## 3. Re-extraction 테스트 (충돌 해결)

In [5]:
async def test_re_extraction():
    print("🔄 Setting Re-extraction 테스트...")
    state = create_base_state()
    state["retry_count"] = 1
    
    # 이전 추출 결과 (불완전한 데이터)
    state["extracted_settings"] = [
        {
            "setting_id": "loc_forest_01",
            "name": "숲",  # 너무 간단함
            "location_type": "forest",
            "visual_background": "forest",  # 너무 간단함
            "atmosphere": "dark",
            "time_of_day": "unknown"  # 잘못됨 - night여야 함
        }
    ]
    
    # 충돌 정보
    state["consistency_report"] = {
        "conflicts": [
            {
                "type": "INSUFFICIENT_DETAIL",
                "description": "visual_background이 너무 간단함. 달빛, 안개 등 상세 묘사 필요",
                "severity": "MEDIUM"
            },
            {
                "type": "INCORRECT_TIME",
                "description": "time_of_day가 unknown이지만 텍스트에 '달빛'이 언급됨 = night",
                "severity": "LOW"
            }
        ]
    }
    
    return await setting_extraction_node(state)

re_result = run_async(test_re_extraction())
pretty_print("Re-extraction 결과", re_result)

🔄 Setting Re-extraction 테스트...
[SETTING] Re-extracting with 2 conflicts as feedback

📋 Re-extraction 결과
{
  "extracted_settings": [
    {
      "setting_id": "loc_forest_01",
      "name": "숲",
      "location_type": "forest",
      "visual_background": "forest",
      "atmosphere": "dark",
      "time_of_day": "unknown"
    }
  ],
  "errors": [
    "Setting JSON parse error: Expecting value: line 1 column 1 (char 0)"
  ],
  "messages": [
    {
      "role": "setting_agent",
      "content": "Failed to parse response"
    }
  ]
}


In [6]:
# Re-extraction 분석
print("\n" + "="*70)
print("📊 Re-extraction 분석")
print("="*70)

re_settings = re_result.get("extracted_settings", [])

for setting in re_settings:
    print(f"\n🏰 [{setting.get('setting_id', 'N/A')}] {setting.get('name', 'Unknown')}")
    
    # 검증: visual_background가 더 상세해졌는가?
    visual_bg = setting.get('visual_background', '')
    is_detailed = len(visual_bg) > 20
    print(f"   Visual Background 상세도: {'✅ 충분' if is_detailed else '❌ 부족'} ({len(visual_bg)} chars)")
    
    # 검증: time_of_day가 수정되었는가?
    time = setting.get('time_of_day', '')
    is_correct_time = time == 'night'
    print(f"   Time of Day: {time} {'✅' if is_correct_time else '❌ (night여야 함)'}")


📊 Re-extraction 분석

🏰 [loc_forest_01] 숲
   Visual Background 상세도: ❌ 부족 (6 chars)
   Time of Day: unknown ❌ (night여야 함)


💡 마지막 2%를 채우기 위한 제언 (For High-Quality Image Gen)현재 결과는 "정확"하지만, 이미지 생성 관점에서는 다소 건조(Dry)할 수 있습니다. 텍스트 원문에 숲에 대한 구체적 묘사가 없었기 때문에, LLM이 딱 "Dark Forest" 정도만 뽑아낸 것입니다.하지만 고퀄리티 일러스트를 위해서는 **LLM의 "창의적 추론(Creative Inference)"**을 허용해야 합니다.1. "시각적 디테일 환각(Hallucinate Details)" 허용하기배경 아티스트에게 "숲을 그려"라고 하면, 알아서 이끼, 안개, 달빛을 추가합니다. 에이전트에게도 **"텍스트에 없어도 분위기에 맞는 디테일을 추가해"**라고 지시하세요.프롬프트 추가 문구:"If the text description is too simple (e.g., just 'forest'), CREATIVELY INFER plausible visual details to match the atmosphere. Add textures (e.g., rough bark, mossy rocks), lighting effects (e.g., god rays, rim light), and environmental particles (e.g., fireflies, falling leaves)."예상 결과 변화:Dark forest $\rightarrow$ Ancient dark forest with gnarled roots, thick moss on ground, volumetric fog, blue moonlight filtering through canopy2. 스타일/화풍(Art Style) 필드 추가배경이 실사체인지, 애니메이션 풍인지, 수묵화 풍인지 결정해야 합니다.JSON 필드 추가:JSON"art_style": "Dark Fantasy, Realistic, Cinematic Lighting, 8k resolution"
팁: 이 부분은 User Input으로 받거나, 스토리 장르(무협, 판타지)에 따라 고정값(Hard-coded)으로 박아넣는 것이 시스템적으로 더 안정적일 수 있습니다.